# breweryBizdat
### get business data about the brewery

In [1]:
#import dependencies
import pandas as pd
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect
Base = automap_base()
import requests
from config import gkey
import json
import numpy as np

In [2]:
#create engine and import data from sqlite database
engine = create_engine('sqlite:///../static/data/bredat.sqlite', echo=False)
Base.metadata.create_all(engine)

In [3]:
Base.prepare(engine, reflect=True)

In [4]:
session = Session(engine)

In [5]:
#create dataframe
brew_df = pd.read_sql_query("SELECT id, name, street, city, state, zipcode, latitude, longitude FROM breweries", con=engine)
brew_df

,id,name,street,city,state,zipcode,latitude,longitude
0,5016,Bark Brewing Company,3021 Spring Garden St,Greensboro,North Carolina,27403,36.0620103554184,-79.8436043406271
1,5033,Blowing Rock Brewing Company,152 Sunset Dr,Blowing Rock,North Carolina,28605,36.1327196,-81.677229
2,5035,Blue Ghost Brewing Company,125 Underwood Rd,Fletcher,North Carolina,28732,35.4368388,-82.530580304217
3,5067,Bull Durham Beer Co,409 Blackwell St,Durham,North Carolina,27701,35.9915653,-78.90481567784
4,5090,Dirtbag Ales Brewery and Taproom,3623 Legion Rd,Hope Mills,North Carolina,28348,34.9939752576824,-78.9220969691311
...,...,...,...,...,...,...,...,...
265,5273,Thirsty Monk Brewery,92 Patton Ave,Asheville,North Carolina,28801,35.59410425,-82.5553147520814
266,5283,Triskelion Brewing Company,340 7th Avenue East,Hendersonville,North Carolina,28792,35.3206001746298,-82.4578969903109
267,5293,Valley River Brewery & Eatery,71 Tennessee St,Murphy,North Carolina,28906,35.0889040369639,-84.0352081706441
268,5304,White Street Brewing Company,218 S White St,Wake Forest,North Carolina,27587,35.9774896,-78.5100617


In [6]:
#set base url for api call
base_url = "https://maps.googleapis.com/maps/api/place/textsearch/json"
place_url = "https://maps.googleapis.com/maps/api/place/details/json"

In [7]:
#create columns for new data from api call
brew_df['rating']=''
brew_df['user_ratings_total']=''
brew_df

,id,name,street,city,state,zipcode,latitude,longitude,rating,user_ratings_total
0,5016,Bark Brewing Company,3021 Spring Garden St,Greensboro,North Carolina,27403,36.0620103554184,-79.8436043406271,,
1,5033,Blowing Rock Brewing Company,152 Sunset Dr,Blowing Rock,North Carolina,28605,36.1327196,-81.677229,,
2,5035,Blue Ghost Brewing Company,125 Underwood Rd,Fletcher,North Carolina,28732,35.4368388,-82.530580304217,,
3,5067,Bull Durham Beer Co,409 Blackwell St,Durham,North Carolina,27701,35.9915653,-78.90481567784,,
4,5090,Dirtbag Ales Brewery and Taproom,3623 Legion Rd,Hope Mills,North Carolina,28348,34.9939752576824,-78.9220969691311,,
...,...,...,...,...,...,...,...,...,...,...
265,5273,Thirsty Monk Brewery,92 Patton Ave,Asheville,North Carolina,28801,35.59410425,-82.5553147520814,,
266,5283,Triskelion Brewing Company,340 7th Avenue East,Hendersonville,North Carolina,28792,35.3206001746298,-82.4578969903109,,
267,5293,Valley River Brewery & Eatery,71 Tennessee St,Murphy,North Carolina,28906,35.0889040369639,-84.0352081706441,,
268,5304,White Street Brewing Company,218 S White St,Wake Forest,North Carolina,27587,35.9774896,-78.5100617,,


In [8]:
#hit google places api

for index, row in brew_df.iterrows():
    params = {
        "key": gkey,
        "query": f"brewery",
        "location":f"{row.latitude},{row.longitude}"
    }
    
    response = requests.get(base_url, params=params)
    place = response.json()['results'][0]
        
    try:
        brew_df.at[index, 'rating'] = place['rating']
        brew_df.at[index, 'user_ratings_total'] = place['user_ratings_total']

    except:
        pass

In [9]:
brew_df

,id,name,street,city,state,zipcode,latitude,longitude,rating,user_ratings_total
0,5016,Bark Brewing Company,3021 Spring Garden St,Greensboro,North Carolina,27403,36.0620103554184,-79.8436043406271,3.5,4
1,5033,Blowing Rock Brewing Company,152 Sunset Dr,Blowing Rock,North Carolina,28605,36.1327196,-81.677229,4.2,241
2,5035,Blue Ghost Brewing Company,125 Underwood Rd,Fletcher,North Carolina,28732,35.4368388,-82.530580304217,4.7,206
3,5067,Bull Durham Beer Co,409 Blackwell St,Durham,North Carolina,27701,35.9915653,-78.90481567784,4.2,6
4,5090,Dirtbag Ales Brewery and Taproom,3623 Legion Rd,Hope Mills,North Carolina,28348,34.9939752576824,-78.9220969691311,4.7,160
...,...,...,...,...,...,...,...,...,...,...
265,5273,Thirsty Monk Brewery,92 Patton Ave,Asheville,North Carolina,28801,35.59410425,-82.5553147520814,4.5,986
266,5283,Triskelion Brewing Company,340 7th Avenue East,Hendersonville,North Carolina,28792,35.3206001746298,-82.4578969903109,4.7,166
267,5293,Valley River Brewery & Eatery,71 Tennessee St,Murphy,North Carolina,28906,35.0889040369639,-84.0352081706441,4.4,696
268,5304,White Street Brewing Company,218 S White St,Wake Forest,North Carolina,27587,35.9774896,-78.5100617,4.6,185


In [10]:
# add data to sqlite database
brew_df.to_sql('ratings', con=engine)

In [11]:
#test sqlite data
engine.execute("SELECT * FROM ratings").fetchall()[0]

(0, 5016, 'Bark Brewing Company', '3021 Spring Garden St', 'Greensboro', 'North Carolina', '27403', '36.0620103554184', '-79.8436043406271', '3.5', 4)